In [1]:
import os
os.chdir('../')

In [3]:
import rdflib
g = rdflib.Graph()
result = g.parse('assets/envo.owl')

In [ ]:
for node in g:
    print(node[1].description)
    break

In [ ]:
from rdflib.namespace import RDF, OWL, RDFS, Namespace
result = g.query('''
SELECT DISTINCT ?ref ?labelr ?about ?refp ?o WHERE {
	?ref ?refp ?o .
	FILTER ( ?refp IN ( owl:equivalentClass, rdfs:subClassOf ) ) .
    OPTIONAL {
		?ref rdfs:label ?labelr
		FILTER(LANG(?labelr) = "" || LANG(?labelr) = 'en')
	} .
	OPTIONAL {
		?ref obo:IAO_0000115 ?about
	} .
}
''', initNs={ 'rdf': RDF, 'rdfs': RDFS, 'owl': OWL, 'obo': Namespace('http://purl.obolibrary.org/obo/')})

In [ ]:
rows = []
for item in result:
    rows.append([i.toPython() if i is not None else i for i in item])

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns=["subject_id", "subject_label", "subject_about", "relation", "object_id"])

In [ ]:
df[~df.subject_about.isna()]

In [ ]:
df['text'] = df.subject_label + '.' + df.subject_about.apply(lambda x: x if x is not None else '')

In [ ]:
import srsly

srsly.write_jsonl(
    "data/envo.jsonl",
    df.drop_duplicates(["subject_id"])[["subject_id", "subject_label", "subject_about"]]
    .rename(
        {"subject_id": "id", "subject_label": "title", "subject_about": "text"},
        axis=1,
    )
    .fillna("")
    .to_dict("records"),
)

df.drop_duplicates(["subject_id"])["subject_label"].rename({"subject_id": "id"}).to_csv(
    "data/envo_concepts.txt", index=False, header=False
)


In [ ]:
nlp.tokenizer.pipe(['hello', 'world'])

In [ ]:
labels = df.subject_label.drop_duplicates().str.lower()